In [2]:
from pytfa.io.json import load_json_model
from skimpy.io.yaml import load_yaml_model
from skimpy.analysis.oracle.load_pytfa_solution import load_fluxes, \
    load_concentrations, load_equilibrium_constants
from skimpy.sampling.simple_parameter_sampler import SimpleParameterSampler
from skimpy.core import *
from skimpy.mechanisms import *
from scipy.linalg import eigvals as eigenvalues
from sympy import Symbol
from skimpy.core.parameters import ParameterValues,ParameterValuePopulation
import pandas as pd
import numpy as np
import pickle

import helper as hp
from skimpy.core.parameters import load_parameter_population

#sys.path.append("..")


#Convenience factors
NCPU = 32
CONCENTRATION_SCALING = 1e9  # 1 mol to 1 mmol
TIME_SCALING = 1  # 1hr
DENSITY = 1105  # g/L
GDW_GWW_RATIO = 0.3  # Assumes 70% Water
flux_scaling_factor = 1e-3 * (GDW_GWW_RATIO * DENSITY) * CONCENTRATION_SCALING / TIME_SCALING


In [6]:
exp_id = 'fdp1'

met_model = 'ecoli_shikki_regulation'
# names_km = hp.load_pkl(f'models/{met_model}/parameter_names_km_fdp1.pkl')
#ss_idx = 1712

base = f'../models/{met_model}'
path_to_tmodel = f'{base}/thermo/varma_{exp_id}'
path_to_kmodel = f'{base}/kinetic/kin_varma_curated.yaml'
path_to_samples = f'{base}/steady_states/tfa_samples.csv'
path_to_output = f'output/'

tmodel = load_json_model(path_to_tmodel)
kmodel = load_yaml_model(path_to_kmodel)

2024-02-23 15:09:29,836 - thermomodel_new - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


hill_coefficient_ATPM
k1_bwd_ACALDtex
k1_bwd_ACtex
k1_bwd_AKGtex
k1_bwd_CO2tex
k1_bwd_D-LACtex
k1_bwd_ETOHtex
k1_bwd_FORtex
k1_bwd_GLCtex
k1_bwd_GLUtex
k1_bwd_Htex
k1_bwd_MALtex
k1_bwd_NH4tex
k1_bwd_O2tex
k1_bwd_PHEtex
k1_bwd_PYRtex
k1_bwd_SERtex
k1_bwd_SUCCtex
k1_bwd_TRPtex
k1_bwd_TYRtex
k1_fwd_ACALDtex
k1_fwd_ACtex
k1_fwd_AKGtex
k1_fwd_CO2tex
k1_fwd_D-LACtex
k1_fwd_ETOHtex
k1_fwd_FORtex
k1_fwd_GLCtex
k1_fwd_GLUtex
k1_fwd_Htex
k1_fwd_MALtex
k1_fwd_NH4tex
k1_fwd_O2tex
k1_fwd_PHEtex
k1_fwd_PYRtex
k1_fwd_SERtex
k1_fwd_SUCCtex
k1_fwd_TRPtex
k1_fwd_TYRtex
k2_bwd_ACALDtex
k2_bwd_ACtex
k2_bwd_AKGtex
k2_bwd_CO2tex
k2_bwd_D-LACtex
k2_bwd_ETOHtex
k2_bwd_FORtex
k2_bwd_GLCtex
k2_bwd_GLUtex
k2_bwd_Htex
k2_bwd_MALtex
k2_bwd_NH4tex
k2_bwd_O2tex
k2_bwd_PHEtex
k2_bwd_PYRtex
k2_bwd_SERtex
k2_bwd_SUCCtex
k2_bwd_TRPtex
k2_bwd_TYRtex
k2_fwd_ACALDtex
k2_fwd_ACtex
k2_fwd_AKGtex
k2_fwd_CO2tex
k2_fwd_D-LACtex
k2_fwd_ETOHtex
k2_fwd_FORtex
k2_fwd_GLCtex
k2_fwd_GLUtex
k2_fwd_Htex
k2_fwd_MALtex
k2_fwd_NH4tex
k2_f

2024-02-23 15:09:33,251 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


In [7]:
# NEW STUFF #
from skimpy.io.regulation import load_enzyme_regulation
# Add regulation data to kinetic model
path_to_regulation_data = f'{base}/allosteric_regulations.csv'
df = pd.read_csv(path_to_regulation_data)
df_regulations_all = df[df['reaction_id'].isin(list(kmodel.reactions.keys()))]
df_regulations_all = df_regulations_all[df_regulations_all['regulator'].isin(list(kmodel.reactants.keys()))]

print(kmodel.reactions['ACt2rpp'].modifiers)

kmodel = load_enzyme_regulation(kmodel, df_regulations_all)


TabDict([('DSM__h_c', <skimpy.core.modifiers.DisplacementSmallMoleculeModifier object at 0x7f6aa1562828>), ('DSM__h_p', <skimpy.core.modifiers.DisplacementSmallMoleculeModifier object at 0x7f6aa1562cf8>)])


In [8]:
kmodel.prepare()
kmodel.compile_jacobian(sim_type=QSSA, ncpu=NCPU)

ss_idx = 1712 #[1715,1717,2392,2482,3927,4230,4241,4456, 4468]
samples = pd.read_csv(path_to_samples, header=0, index_col=0).iloc[ss_idx, 0:]

# Load ss fluxes and concentrations
flux_series = load_fluxes(samples, tmodel, kmodel,
                  density=DENSITY,
                  ratio_gdw_gww=GDW_GWW_RATIO,
                  concentration_scaling=CONCENTRATION_SCALING,
                  time_scaling=TIME_SCALING)

conc_series = load_concentrations(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING)
# Fetch equilibrium constants
k_eq = load_equilibrium_constants(samples, tmodel, kmodel,
                          concentration_scaling=CONCENTRATION_SCALING,
                          in_place=True)

symbolic_concentrations_dict = {Symbol(k): v for k, v in conc_series.items()}

sampling_parameters = SimpleParameterSampler.Parameters(n_samples=1)
sampler = SimpleParameterSampler(sampling_parameters)

sampler._compile_sampling_functions(kmodel, symbolic_concentrations_dict, [])
model_param = kmodel.parameters

2024-02-23 15:09:40,597 - Unnamed - WARNING - Non integer stoichiometries found ['LMPD_biomass_c_1_420', 'CYTBDpp', 'CYTBO3_4pp'] change to integer for linear dependencies


In [9]:
param_pop = []
max_eigens = []
all_eigens = []
#path = f'output/rnsc_opt/all_integration_test/{rep}/'

#     gen_params = np.load(f'{path}gen_{ix}.npy')
parameter_df = pd.read_csv('./sample_parameters.csv', index_col=0)
parameter_set = pd.DataFrame([parameter_df['parameter_set_0'].values], columns=parameter_df.index)


for j in range(len(parameter_set.index)):
    param_val = parameter_set.loc[j] * CONCENTRATION_SCALING
    param_val = ParameterValues(param_val, kmodel)
    
    kmodel.parameters = k_eq
    kmodel.parameters = param_val
    parameter_sample = {v.symbol: v.value for k, v in kmodel.parameters.items()}
    # Set all vmax/flux parameters to 1.
    # TODO Generalize into Flux and Saturation parameters
    for this_reaction in kmodel.reactions.values():
        vmax_param = this_reaction.parameters.vmax_forward
        parameter_sample[vmax_param.symbol] = 1
        # Calculate the Vmax's

    kmodel.flux_parameter_function(
        kmodel,
        parameter_sample,
        symbolic_concentrations_dict,
        flux_series
    )
    for c in conc_series.index:
        if c in model_param:
            c_sym = kmodel.parameters[c].symbol
            parameter_sample[c_sym] = conc_series[c]

    this_jacobian = kmodel.jacobian_fun(flux_series[kmodel.reactions],
                                        conc_series[kmodel.reactants], parameter_sample)

    # New Code
    real_eigenvalues = np.real(eigenvalues(this_jacobian.todense()))
    max_eigens.append(np.max(real_eigenvalues))
    all_eigens.append(sorted(real_eigenvalues))

    this_param_sample = ParameterValues(parameter_sample, kmodel)
    param_pop.append(this_param_sample)

    del(this_param_sample)
    del(param_val)
    del(this_jacobian)

param_pop2 = ParameterValuePopulation(param_pop, kmodel=kmodel)
param_pop2.save(f'{path_to_output}/gen_0.hdf5')
pd.DataFrame(max_eigens).to_csv(f'{path_to_output}/max_eigens.csv')
pd.DataFrame(all_eigens).to_csv(f'{path_to_output}/all_eigens.csv')


In [11]:
all_eigens

[[-580375455759.9265,
  -177774943012.8822,
  -57616667244.697975,
  -19015930656.146374,
  -2461799024.699238,
  -1477762235.6888943,
  -413684536.9432719,
  -385849113.2313783,
  -171744110.73531693,
  -168801882.93390203,
  -101265625.8210137,
  -62702185.20536425,
  -15163543.4469576,
  -10103630.74164076,
  -9181456.513416814,
  -8750876.78669759,
  -7858279.00362444,
  -7172240.013745409,
  -6527975.201046003,
  -6025301.552335497,
  -5560099.345535646,
  -5067296.993328907,
  -5044791.790196428,
  -4181730.9055480435,
  -3962746.8070850633,
  -3833912.060395379,
  -3422515.2888610205,
  -2466553.557272546,
  -1221208.2063450024,
  -887885.9416790146,
  -514377.7165558068,
  -447680.1032831602,
  -377503.7748289436,
  -355179.6029644016,
  -336333.2015931817,
  -335215.0135614024,
  -202925.52645839128,
  -158319.11959085366,
  -138815.39638052267,
  -117436.4499106729,
  -100583.01584017121,
  -81281.04642342527,
  -51726.90310700812,
  -28592.72071299137,
  -26479.908298643328,